## Stub Plans 

### Links Úteis

- Documentação dos plan stubs: https://blueskyproject.io/bluesky/main/plans.html#stub-plans

- Documentação da customização de planos com per_step: https://blueskyproject.io/bluesky/main/plans.html#customize-step-scans-with-per-step

- Documentação da utilização de metadados: https://blueskyproject.io/bluesky/main/metadata.html

- Documentação dos preprocessors: https://blueskyproject.io/bluesky/main/plans.html#plan-preprocessors

In [136]:
%matplotlib qt
from time import time
from bluesky import RunEngine
from bluesky.plans import count
from bluesky.callbacks import LiveTable
from bluesky.plan_stubs import abs_set, rel_set, mv, mvr, \
    stage, unstage, stop, pause, deferred_pause, sleep, \
    checkpoint, clear_checkpoint, repeat, trigger_and_read, \
    open_run, close_run
from ophyd import Device, Component, ADComponent
from ophyd.sim import noisy_det, motor1, motor2, SynGauss, FakeEpicsSignalWithRBV
from matplotlib.pyplot import ion
from bluesky.utils import install_nb_kicker
install_nb_kicker()

ion()
RE = RunEngine()

#### Getting values from ophyd objects

In [ ]:
print(f"Read: {motor1.read()}")
print()
print(f"Configuration: {motor1.read_configuration()}")
print()
print(f"Get Device: {motor1.get()}")
print()
print(f"Get Readback: {motor1.readback.get()}")

## mv and abs_set

In [ ]:
#help(abs_set)
#help(mv)

In [ ]:
RE(mv(motor1, 0))
motor1.delay = 2
print(f"Before: {motor1.read()}")
RE(abs_set(motor1, 15))
print(f"After: {motor1.read()}")
motor1.delay = 0

In [ ]:
RE(mv(motor1, 0))
motor1.delay = 2
print(f"Before: {motor1.read()}")
RE(abs_set(motor1, 15, wait=True))
print(f"After: {motor1.read()}")
motor1.delay = 0

In [ ]:
RE(mv(motor1, 0, motor2, 0))
motor1.delay = 2
motor2.delay = 4
print(f"Before : {motor1.readback.get()}")
print(f"Before motor2: {motor1.readback.get()}")
start = time()
RE(mv(motor1, 4, motor2, 15))
end = time()
print(f"After motor1: {motor1.readback.get()}")
print(f"After motor2: {motor2.readback.get()}")
print(f"Duration: {end - start}")
motor1.delay = 0
motor2.delay = 0

In [128]:
class SimPimegaCam(Device):
    
    value = ADComponent(SynGauss, motor=motor1, motor_field="motor1", center=160, 
        Imax=20, sigma=100, noise="uniform", noise_multiplier=0.1, kind="hinted")
    trigger_mode = ADComponent(FakeEpicsSignalWithRBV, "TriggerMode", string=True, kind="config")
    file_name = ADComponent(FakeEpicsSignalWithRBV, "FileName", string=True, kind="config")
    file_path = ADComponent(FakeEpicsSignalWithRBV, "FilePath", string=True, kind="config")

    file_template = ADComponent(FakeEpicsSignalWithRBV, "FileTemplate", string=True, kind="config")
    auto_increment = ADComponent(FakeEpicsSignalWithRBV, "AutoIncrement", string=True, kind="config")
    auto_save = ADComponent(FakeEpicsSignalWithRBV, "AutoSave", string=True, kind="config")
    
    num_capture = ADComponent(FakeEpicsSignalWithRBV, "NumCapture", kind="config")
    num_exposures = ADComponent(FakeEpicsSignalWithRBV, "NumExposures", kind="config")
    
    acquire_time = ADComponent(FakeEpicsSignalWithRBV, "AcquireTime", kind="config")
    acquire_period = ADComponent(FakeEpicsSignalWithRBV, "AcquirePeriod", kind="config")

In [113]:
def pimega_configuration(
        pimega_cam: any, acquire_time: float, num_exposures: int,
        file_name: str, file_path: str, det_readout: float = 0.01):
    
    yield from mv(pimega_cam.auto_save, "Yes")
    yield from mv(pimega_cam.auto_increment, "Yes")
    yield from mv(pimega_cam.file_template, "%s%s_%3.3d.hdf5")
    yield from mv(pimega_cam.trigger_mode, "External")

    yield from mv(pimega_cam.num_capture, num_exposures)
    yield from mv(pimega_cam.num_exposures, num_exposures)
    
    yield from mv(pimega_cam.num_capture, num_exposures)
    yield from mv(pimega_cam.num_exposures, num_exposures)
        
    yield from mv(pimega_cam.acquire_time, acquire_time)
    yield from mv(pimega_cam.acquire_period, acquire_time + det_readout)
    
    yield from mv(pimega_cam.file_name, file_name)
    yield from mv(pimega_cam.file_path, file_path)

In [114]:
def get_values():
    attr_dict = {}
    for attr_name, values in pimega_cam.read_configuration().items():
        attr_dict[attr_name] = values["value"]
    return attr_dict

pimega_cam = SimPimegaCam("SIM:", name="pimega")

before_config = get_values()
RE(pimega_configuration(pimega_cam, 0.1, 15, "acquisiton_test", "/ibira/test"))
after_config = get_values()

for key in after_config.keys():
    print(f"{key}: {before_config[key]} -> {after_config[key]}")

pimega_trigger_mode: 0 -> External
pimega_file_name: 0 -> acquisiton_test
pimega_file_path: 0 -> /ibira/test
pimega_file_template: 0 -> %s%s_%3.3d.hdf5
pimega_auto_increment: 0 -> Yes
pimega_auto_save: 0 -> Yes
pimega_num_capture: 0 -> 15
pimega_num_exposures: 0 -> 15
pimega_acquire_time: 0 -> 0.1
pimega_acquire_period: 0 -> 0.11


## mvr and rel_set

In [ ]:
#help(mvr)
#help(rel_set)

In [ ]:
RE(mv(motor1, 14))
motor1.delay = 2
print(f"Before: {motor1.readback.get()}")
RE(rel_set(motor1, 15))
print(f"After: {motor1.readback.get()}")
motor1.delay = 0

In [ ]:
RE(mv(motor1, 14))
motor1.delay = 2
print(f"Before: {motor1.readback.get()}")
RE(mvr(motor1, 15))
print(f"After +15: {motor1.readback.get()}")
RE(mvr(motor1, -10))
print(f"After -10: {motor1.readback.get()}")
RE(mvr(motor1, 2.5))
print(f"After +2.5: {motor1.readback.get()}")
motor1.delay = 0

In [131]:
def count_and_rotate(detector: any, rx: any, ux: any):
    yield from mv(rx, 0)
    
    for delta_angle in range(0, 10):
        yield from scan([pimega_cam.value], ux, -1, 1, 5)
        yield from mvr(rx, 36)
        
        print(f"Motor Rx: {rx.readback.read()}")

In [135]:
pimega_cam = SimPimegaCam("SIM:", name="pimega")

RE(count_and_rotate(pimega_cam, motor1, motor2), LiveTable(['pimega_value']))



+-----------+------------+--------------+
|   seq_num |       time | pimega_value |
+-----------+------------+--------------+
|         1 | 15:02:36.5 |        5.489 |
|         2 | 15:02:36.5 |        5.626 |
|         3 | 15:02:36.6 |        5.479 |
|         4 | 15:02:36.6 |        5.657 |
|         5 | 15:02:36.6 |        5.633 |
+-----------+------------+--------------+
generator scan ['70d10a90'] (scan num: 31)


Motor Rx: {'motor1': {'value': 36.0, 'timestamp': 1725645756.793538}}


+-----------+------------+--------------+
|   seq_num |       time | pimega_value |
+-----------+------------+--------------+
|         1 | 15:02:36.8 |        9.304 |
|         2 | 15:02:36.9 |        9.180 |
|         3 | 15:02:36.9 |        9.307 |
|         4 | 15:02:36.9 |        9.254 |
|         5 | 15:02:37.0 |        9.202 |
+-----------+------------+--------------+
generator scan ['2e18acf8'] (scan num: 32)


Motor Rx: {'motor1': {'value': 72.0, 'timestamp': 1725645757.1499088}}


+------

('70d10a90-fa0b-4bb4-aaaf-0805cc87ecb3',
 '2e18acf8-5909-49e0-b909-9a5f267c4e13',
 'adcbc326-8642-4925-84fa-ef4aeaaafca7',
 'a52c7fdb-f6be-439c-b140-32fcceac2506',
 'e4806b16-0745-49d5-a300-ac910368d987',
 '1466a452-b82b-4064-be23-d53e3eb8a6d2',
 'dcd07b5f-84d2-4509-9c48-88f3158308f9',
 'e835f2ec-d6bf-4c5d-a5cc-eeb23000f2bf',
 'a4b89a1a-5c26-4e77-b699-3ffdef4d4490',
 '1b243374-39e0-4083-a474-650df7e8d6a2')

## sleep and stop

In [ ]:
#help(sleep)
#help(stop)

In [ ]:
class StopDevice(Device):

    def stop(self):
        super().stop()
        print("This Device has been stopped")

In [ ]:
stop_device = StopDevice(name="stop_device")
RE(stop(stop_device))

In [ ]:
start = time()
RE(sleep(1))
end = time()
print(f"Duration: {end - start}")

## stage and unstage

In [ ]:
#help(stage)
#help(unstage)

In [137]:
class StagedDevice(Device):

    def stage(self):
        super().stage()
        print("This Device is being armed")

    def unstage(self):
        super().unstage()
        print("This Device is being restored to a safe idle state")

In [ ]:
exampleDevice = StagedDevice(name="staged_device")

def stage_unstage_plan():
    yield from stage(exampleDevice)
    
    # Custom plan
    print("Custom plan")
    
    yield from unstage(exampleDevice)

RE(stage_unstage_plan())

## Controlling the Run Engine 

In [ ]:
#help(checkpoint)
#help(clear_checkpoint)
#help(pause)
#help(deferred_pause)
#help(open_run)
#help(close_run)

### Pause

In [ ]:
def pause_plan():
    yield from open_run()
    yield from mv(motor1, 5)
    yield from trigger_and_read([noisy_det])
    yield from checkpoint()
    
    yield from pause()
    
    yield from mv(motor1, 4)
    yield from trigger_and_read([noisy_det])
    yield from checkpoint()
    
    yield from mv(motor1, 14)
    yield from trigger_and_read([noisy_det])
    yield from checkpoint()
    
    yield from close_run()

In [ ]:
RE(pause_plan(), LiveTable(["noisy_det"]))

In [ ]:
print(RE.state)
RE.resume()

### Deferred Pause

In [ ]:
def deferred_pause_plan():
    yield from open_run()
    yield from mv(motor1, 5)
    yield from trigger_and_read([noisy_det])
    yield from checkpoint()
    
    yield from deferred_pause()
    
    yield from mv(motor1, 4)
    yield from trigger_and_read([noisy_det])
    yield from checkpoint()
    
    yield from mv(motor1, 14)
    yield from trigger_and_read([noisy_det])
    yield from checkpoint()
    
    yield from close_run()

In [ ]:
RE(deferred_pause_plan(), LiveTable(["noisy_det"]))

In [ ]:
print(RE.state)
RE.resume()

### clear_checkpoint

In [ ]:
def clear_checkpoint_plan():
    yield from open_run()
    yield from mv(motor1, 5)
    yield from trigger_and_read([noisy_det])
    yield from clear_checkpoint()
    
    yield from pause()
    
    yield from mv(motor1, 4)
    yield from trigger_and_read([noisy_det])
    yield from checkpoint()
    
    yield from mv(motor1, 14)
    yield from trigger_and_read([noisy_det])
    yield from checkpoint()
    
    yield from close_run()

In [ ]:
RE(clear_checkpoint_plan(), LiveTable(["noisy_det"]))

In [ ]:
RE.state

## repeat

In [ ]:
help(repeat)

In [ ]:
def repeat_count():
    yield from count([noisy_det], num=5, delay=0.25)
RE(repeat(repeat_count, num=3, delay=3), LiveTable(["noisy_det"]))